# Imports and Authorizations

In [ ]:
# Imports used in this notebook.
import ee
import os
import sys
import time
import shutil
import pandas as pd
import seaborn as sea
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive

# Must authenticate your EE account before use of the package.
ee.Authenticate()
ee.Initialize()

# This is how we can access our drive to get the correlation product.
drive.mount('/content/gdrive')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nmcpAVXAguft9ShDzkDvKUu9pW7Q26YKY7QQ03tE8jU&tc=cInW51HqzNLknrcXDNBt4YaZcgappXFu-HbTGPv0uoo&cc=vXImGzZxgst2OH78rt751WQpnk6YOKrtMpopxVD-j0A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNiO9Ku3x2YST5yE-FlAxOzVX-RnqDfDTvpVcpyPhUw1RyrxlWWzaU

Successfully saved authorization token.
Mounted at /content/gdrive


# Set the Biscayne Bay AOI.

In [ ]:
#Florida DEP bounds
#Indian River Lagoon
siteName = 'Indian River Lagoon'
siteName_2char = 'IRL'
upperLeft = [-80.763739, 28.621909]
lowerRight = [-80.503601,  27.605387]
# upperLeft = [-80.763739, 28.621909]
# lowerRight = [-80.533771,   28.063011]
# Define a region of interest to filter our collection.
roi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1])

# Create a list of dates to use for image extraction

In [ ]:
# List of dates.
date_list = [
# ee.Date('2020-01-01'),
# ee.Date('2020-01-06'),
# ee.Date('2020-01-16'),
# ee.Date('2020-02-03'),
# ee.Date('2020-02-08'),
# ee.Date('2020-03-01'),
# ee.Date('2020-03-14'),
# ee.Date('2020-03-24'),
# ee.Date('2020-03-29'),
# ee.Date('2020-04-03'),
# ee.Date('2020-04-08'),
# ee.Date('2020-05-05'),
# ee.Date('2020-05-20'),
# ee.Date('2020-06-24'),
# ee.Date('2020-06-29'),
# ee.Date('2020-07-14'),
# ee.Date('2020-07-19'),
# ee.Date('2020-07-24'),
# ee.Date('2020-08-08'),
# ee.Date('2020-08-13'),
# ee.Date('2020-09-02'),
# ee.Date('2020-09-17'),
# ee.Date('2020-10-15'),
# ee.Date('2020-10-27'),
# ee.Date('2020-11-24'),
# ee.Date('2020-12-09'),
# ee.Date('2020-12-21'),
# ee.Date('2020-12-29'),
# ee.Date('2021-01-05'),
# ee.Date('2021-01-20'),
# ee.Date('2021-01-28'),
# ee.Date('2021-02-02'),
# ee.Date('2021-02-19'),
# ee.Date('2021-02-22'),
# ee.Date('2021-02-24'),
# ee.Date('2021-02-27'),
# ee.Date('2021-03-01'),
# ee.Date('2021-03-04'),
# ee.Date('2021-03-11'),
# ee.Date('2021-03-14'),
# ee.Date('2021-03-19'),
# ee.Date('2021-03-24'),
# ee.Date('2021-04-05'),
# ee.Date('2021-04-10'),
# ee.Date('2021-04-30'),
# ee.Date('2021-05-05'),
# ee.Date('2021-05-10'),
# ee.Date('2021-05-15'),
# ee.Date('2021-05-25'),
# ee.Date('2021-06-09'),
ee.Date('2021-07-19'),
# ee.Date('2021-07-29'),
# ee.Date('2021-09-07'),
# ee.Date('2021-09-22'),
# ee.Date('2021-09-27'),
# ee.Date('2021-09-30'),
# ee.Date('2021-10-02'),
# ee.Date('2021-10-12'),
# ee.Date('2021-11-16'),
# ee.Date('2021-11-26'),
# ee.Date('2021-11-29'),
# ee.Date('2021-12-04'),
# ee.Date('2021-12-06'),
# ee.Date('2021-12-09'),
# ee.Date('2021-12-16'),
# ee.Date('2021-12-24'),
# ee.Date('2021-12-26'),
# ee.Date('2021-12-29'),
# ee.Date('2022-01-03'),
# ee.Date('2022-01-08'),
# ee.Date('2022-01-15'),
# ee.Date('2022-01-18'),
# ee.Date('2022-01-20'),
# ee.Date('2022-01-30'),
# ee.Date('2022-02-04'),
# ee.Date('2022-02-12'),
# ee.Date('2022-02-14'),
# ee.Date('2022-02-22'),
# ee.Date('2022-02-24'),
# ee.Date('2022-03-04'),
# ee.Date('2022-03-21'),
# ee.Date('2022-03-26'),
# ee.Date('2022-04-05'),
# ee.Date('2022-04-10'),
# ee.Date('2022-04-25'),
# ee.Date('2022-05-05'),
# ee.Date('2022-05-15'),
# ee.Date('2022-06-24'),
# ee.Date('2022-07-19'),
# ee.Date('2022-07-29'),
# ee.Date('2022-08-01'),
# ee.Date('2022-08-03'),
# ee.Date('2022-08-18'),
# ee.Date('2022-09-07'),
# ee.Date('2022-09-12'),
# ee.Date('2022-09-22'),
# ee.Date('2022-10-02'),
# ee.Date('2022-10-05'),
# ee.Date('2022-10-07'),
]

# Select our Sentinel-2 imagery


In [ ]:
im_list = []

# Start Standard code for processing images from a collection.
for date in date_list:

  # Bring in our imagery.
  coll = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(roi)
              .filterDate(date, date.advance(1,'day')))

  # Clip our input image to our region of interest.
  # Set the name of the original image to our new image.
  water = coll.mosaic().clip(roi).set({'name':coll.first().get('system:index')})

  # Add the newly created image to the empty list created above.
  im_list += [water]

# Begin analysis

In [ ]:
# Name the parent folder where we're writing our output files.
# output_folder = '/content/gdrive/Shared drives/Asgard/FL_DEP_Seagrass_KSU_ST/Sentinel2/part2'
output_folder = '/content/gdrive/My Drive/test_FL_DEP/part2'

# Define the Sentinel-2 bands to extract
non_ee_band_list = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A']
# For reductions.
native_res = 10
maxP = 1e10

# Array of wavelength corrections for Sentinel-2 sensor.
wl = ee.Array([443.9,496.6,560.0,664.5,703.9,740.2,782.5,835.1,864.8])
# wl = ee.Array([442.7,492.7,559.8,664.4,704.1,740.5,782.8,832.8,864.7])

# Name the band combo for outputs.
bandCombo="B1-B8A"

""" Bands and Wavelength Array to select. You can add or subtract any you'd like here and the code
    below will account for the change throughout the workflow. """

# Use this for the derivative calculation.
band_list_early_length = len(non_ee_band_list)
# We use this everywhere else after the derivative calc (original band list x2).
#band_list_length = band_list_early_length * 2
band_list_length = band_list_early_length
# Use this in a couple for loops throughout the code.
band_list_forLoops = range(band_list_length)

In [ ]:
# Use this for the commented sections in the code block below.
avg_bandVals = '/content/gdrive/My Drive/test_FL_DEP/s2_avgWeightedBandValues.csv'

# Here are our weighted, average band values.
bv_pd = pd.read_csv(avg_bandVals, delimiter = ',').drop('Unnamed: 0', axis = 1)['0'].tolist()

weighted_band_image = ee.ImageCollection([ee.Image(value) for value in range(band_list_early_length)]).toBands()

In [ ]:
def derivative_images(water):

  """This section is devoted to modifying the image, making it suitable for
     the Derivative Spectra function, and running the DS function."""

  # Parse out the water pixels in Sentinel 2 A/B MSI image.
  # This function masks the cloudy pixels and identifies the water pixels.
  def get_water(image):

    # Water with cloud check, but note this is the not bit addition method.
    water_pixels = (image.select('QA60').eq(0).And(image.select('SCL').eq(6)))
    # attempts at masking better
    # water_pixels=image.select('QA60')
    # cloudBitMask=1 << 10
    # cirrusBitMask=1 << 11
    # mask = water_pixels.bitwiseAnd(cloudBitMask).eq(0).And(water_pixels.bitwiseAnd(cirrusBitMask).eq(0))
    # Mask all non-water pixels in the image.
    image = image.updateMask(water_pixels)

    return image.select(non_ee_band_list).toDouble()

  # Run the S2 mask on the test images.
  water_mask = get_water(water)

  # Using this for the subtraction of average values, addition of weighted average values.

  # Making an image out of the average values. This creates a constant image.
  band_averages = water_mask.reduceRegion(ee.Reducer.mean(),roi,maxPixels = maxP, scale = native_res,bestEffort=True).values().getInfo()

  single_image_averages = ee.ImageCollection([ee.Image(value) for value in band_averages]).toBands()

  # Update the image by first subtracting individual image average, then by adding on the weighted average band values.
  water_mask = water_mask.subtract(single_image_averages).add(weighted_band_image)

  # Binary reducer, which returns 1 everywhere that water hasn't been masked.
  mask_image = water_mask.reduce(ee.Reducer.anyNonZero())

  # Any pixels passing the pixel mask will now be polygonized.
  water_geom = mask_image.reduceToVectors(scale = native_res, geometry = roi, maxPixels = maxP, bestEffort = True)

  # This function will select the bands of importance and clip the raster.
  def Clip(img, geo):
    return img.clip(geo)

  clipped_image = Clip(water_mask,water_geom)

  # Add code below to divide the reflectance bands by 10^4 to scale from 0 to 1
  clipped_image = clipped_image.multiply(0.0001)

  def derivativeSpectra(img,numBands):

    drdlraster = None

    for i in range(numBands):

      # Special case of the equation is required for the first band.
      if i == 0:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i])
          }).rename('image_Index' + str(i) + '_derivative')

      # Special case of the equation is required for the last band.
      elif i == (numBands-1):

        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Apply equation for all other bands in the image.
      else:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Adds derivative bands to the input image.
      img = img.addBands(drdlraster)

    # Final image has the name of its original image reassigned.
    # This is used for file naming conventions later on.
    return img.set({'name': water.get('name')}).select(8,9,10,11,12,13,14,15,16)

  # Run the Derivative Spectra function on the test image.
  d_image = derivativeSpectra(clipped_image,band_list_early_length)

  return d_image

  # ======================================================== #

In [ ]:
# Make derivative images for all of our images of interest.
der_list = [derivative_images(ee.Image(im)) for im in im_list]

# Priming for VPCA

In [ ]:
# This code will be used for making some subfolders and writing image data to said folders.

# Length of our derivative list.
image_list_length = len(der_list)

# Using the variables created above, get the names of the images that we are using in our analysis.
image_names = [ee.Image(der_list[i]).get('name').getInfo() for i in range(image_list_length)]

In [ ]:
image_names

['20210719T160519_20210719T160513_T17RNL']

In [ ]:
# Make the part2 folder.
os.mkdir(output_folder)

FileExistsError: ignored

In [ ]:
# This is where each respective image's information will be saved.
# Notice that we're using the image names we just made.
for name in image_names:
  os.mkdir(f'{output_folder}/{name}_outputs')

In [ ]:
# List our directory of folders, which we can utilize to write files.
output_folders = os.listdir(output_folder)

In [ ]:
output_folders

['20200101T160511_20200101T160506_T17RNL_outputs',
 '20220801T155911_20220801T161100_T17RNL_outputs',
 '20220803T160519_20220803T160516_T17RNL_outputs',
 '20200106T160509_20200106T160506_T17RNL_outputs',
 '20200116T160509_20200116T160505_T17RNL_outputs',
 '20200314T160021_20200314T160651_T17RNL_outputs',
 '20210224T160511_20210224T160511_T17RNL_outputs',
 '20210719T160519_20210719T160513_T17RNL_outputs']

# Large functions used in the analysis that I've removed for the purpose of shortening the analysis block

In [ ]:
"""Section devoted to performing the Varimax Rotation."""

# This function is used to restructure the array after each pass through the rotation loop.
def vpca_toArray(initArr, a, b, xRot, yRot):
# Values of "i" and "j" are checked to return a new, reformatted version of the input array.
  if a == 0 and b == 1:
    initArr = ee.Array.cat([xRot,yRot,initArr.slice(1,2,3)],1)
  elif a == 0 and b == 2:
    initArr = ee.Array.cat([xRot,initArr.slice(1,1,2),yRot],1)
  elif a == 1 and b == 2:
    initArr = ee.Array.cat([initArr.slice(1,0,1),xRot,yRot],1)
  else:
    initArr = initArr
  return initArr

def varimax_rotation(input, flag = False):

  epsilon = ee.Number(2 * 0.00001)

  # Sqrt of communality.
  # (Communality: the proportion of variance that each item has in common with other items.)
  h = ((input.pow(2)).reduce(reducer=ee.Reducer.sum(),axes=([1]))).sqrt()
  # Extend "h" to match the dimensions of the input array.
  h_b = h.repeat(1,3)
  # Input is normalized by the sqrt of communality.
  bh = input.divide(h_b)

  """======================================================="""
  """Create list to be used for iteration. If you need to change
    the amount of iterations through the rotate() function,
    this is where you do it."""
  # Max seq. on which this step has been tested and worked: (0,6)
  seq = ee.List.sequence(0,6)
  """======================================================="""
  def rotate(current,prev):

    bh = ee.Array(prev)

    for i in range(2):
      for j in range(3):
        #  Select i'th column from the matrix.
        xx = bh.slice(1,i,i+1)
        #  Select the j'th column from the matrix.
        yy = bh.slice(1,j,j+1)

        #  Calculations below are made to determine phi.
        uu = (xx.pow(2)).subtract(yy.pow(2))

        vv = xx.multiply(yy).multiply(2)

        aa = uu.reduce(reducer=ee.Reducer.sum(),axes=([0])).get([0,0])

        bb = vv.reduce(reducer=ee.Reducer.sum(),axes=([0])).get([0,0])

        cc = (uu.pow(2)).subtract(vv.pow(2)).reduce(
                reducer=ee.Reducer.sum(),
                axes = ([0])
                ).get([0,0])

        dd = (uu.multiply(vv).multiply(2)).reduce(
                reducer = ee.Reducer.sum(),
                axes = ([0])
                ).get([0,0])

      num = dd.subtract((aa.multiply(bb).multiply(2)).divide(band_list_length))

      den = cc.subtract(((aa.pow(2)).subtract(bb.pow(2))).divide(band_list_length))

      phi = (den.atan2(num)).divide((band_list_length) - 1)

      # Determine if phi is greater than or equal to epsilon.
      evaluation = (phi.abs()).gte(epsilon)

      # Rotation takes place in server-side if statement.
      rot = ee.Algorithms.If(
                  condition=ee.Number(evaluation).eq(1),
                  trueCase=
                  {
                    'xx_rot': (xx.multiply(phi.cos())).add(yy.multiply((phi.sin()))),
                    'yy_rot': (xx.multiply((phi.multiply(-1)).sin())).add(yy.multiply(phi.cos()))
                  },
                  falseCase=None)
      # Dictionary contains the two rotations.
      rot = ee.Dictionary(rot)

      # Set the new value of "xx".
      setX = ee.Algorithms.If(
              condition=ee.Number(evaluation).eq(1),
              trueCase=ee.Dictionary(rot).getArray('xx_rot'),
              falseCase=xx)
      # Set the new value of "yy".
      setY = ee.Algorithms.If(
              condition=ee.Number(evaluation).eq(1),
              trueCase=ee.Dictionary(rot).getArray('yy_rot'),
              falseCase=yy)

      # Cast "xx" and "yy" into Array objects for the reformation function.
      xx = ee.Array(setX)
      yy = ee.Array(setY)

      # Reform the rotated array.
      bh = vpca_toArray(bh,i,j,xx,yy)

    # Checks the remaining number of pairs, determine if array has converged.
    return bh

  # Iterate through each position of the list defined above.
  seq = seq.iterate(rotate,bh)

  # Cast the result of the rotation as new bh.
  bh = ee.Array(seq)

  # Denormalize bh by multiplying it by the sqrt of communality.
  result = bh.multiply(h_b)

  fracVar = ((result.pow(2)).reduce(ee.Reducer.sum(),([0]))).divide(band_list_length)

  return ee.Dictionary({'result':result,'h':h,'fracVar':fracVar})

# Project original data onto component axis to create component scores.
def create_scores(data):

  # Calculate factor score coefficients.
  trans = data.transpose()
  v = trans.matrixMultiply(data)
  inverse = v.matrixInverse()
  b = data.matrixMultiply(inverse)

  return b

"""EE table export requires a Feature Collection.
   This step reformats the VPCA Loadings into a
   Feature, which can be accepted as an input in
   a Feature Collection."""

# Compile stats into EE Feature (necessary for export).
def createFeats_loads(sliver):
  row1 = sliver.get([0,0])
  row2 = sliver.get([0,1])
  row3 = sliver.get([0,2])
  row4 = sliver.get([0,3])
  row5 = sliver.get([0,4])
  row6 = sliver.get([0,5])
  row7 = sliver.get([0,6])
  row8 = sliver.get([0,7])
  row9 = sliver.get([0,8])
  row10 = sliver.get([0,9])

  rowsForCsv = {
      'Wavelengths': row1,
      'component 1': row2,
      'component 2': row3,
      'component 3': row4,
      'component 4': row5,
      'component 5': row6,
      'component 6': row7,
      'Comm_1-3': row8,
      'Comm_4-6': row9,
      'Comm_tot': row10
      }

  return ee.Feature(None,rowsForCsv)

def createFeats_eigens(sliver, title = None):
  row1 = sliver.get(0)
  row2 = sliver.get(1)
  row3 = sliver.get(2)
  row4 = sliver.get(3)
  row5 = sliver.get(4)
  row6 = sliver.get(5)

  rowsForCsv = {
      'Wavelengths': title,
      'component 1': row1,
      'component 2': row2,
      'component 3': row3,
      'component 4': row4,
      'component 5': row5,
      'component 6': row6,
      'Comm_1-3': 'Total',
      'Comm_4-6': sliver.slice(0,6).reduce(ee.Reducer.sum()),
      'Comm_tot': None
      }

  return ee.Feature(None, rowsForCsv)

def createFeats_fracVar(sliver, title = None):
  row1 = sliver.get([0])
  row2 = sliver.get([1])
  row3 = sliver.get([2])
  row4 = sliver.get([3])
  row5 = sliver.get([4])
  row6 = sliver.get([5])

  rowsForCsv = {
      'Wavelengths': title,
      'component 1': row1,
      'component 2': row2,
      'component 3': row3,
      'component 4': row4,
      'component 5': row5,
      'component 6': row6,
      'Comm_1-3': 'Total',
      'Comm_4-6': sliver.reduce(ee.Reducer.sum(), axes = [0]).get([0]),
      'Comm_tot': None
      }

  return ee.Feature(None, rowsForCsv)

# Full analysis function

In [ ]:
der_list

In [ ]:
def fullAnalysis(fn, image, out_name, out_fold):
  # First, load our csv file into a Pandas DataFrame.
  drive_string = fn
  dataframe = pd.read_csv(drive_string, delimiter = ',').drop('Unnamed: 0',1)

  # Next, convert our dataframe to a numpy array.
  corr_ls = dataframe.to_numpy()

  # Eigenvalues and eigenvectors calculated using Numpy.
  eigens = np.linalg.eigh(corr_ls)

  # Extract the eigenvalues from eigens so that we can reverse the sort
  eigenVals = eigens[0]

  # Reverse the eigenvalue sort so that they are in descending order
  eigenVals = eigenVals[::-1]
  # Convert to an EE list.
  eigenVals = ee.List(list(eigenVals))

  # Build the eigenvector matrix
  # Rotate the eigenvector matrix 180 to place it in the correct orientiation to match
  # the eigenvalues sorted in descending order
  rotating = np.rot90(eigens[1],2)

  # Take each row of our eigenvector matrix and concatenate the rows in a new EE array.
  eigenVecs = ee.Array.cat([ee.Array(list(rotating[i])) for i in range(len(rotating))])

  # Reformat the array. May need to be multiplied by -1.
  eigenVecs = eigenVecs.reshape([band_list_length,band_list_length]).transpose()

  # Checking results...
  # for row in eigenVecs.getInfo():
  #   print(row)

  # ================= #
  """The following section is devoted to pre-processing
   the data for the Varimax Rotation."""

  # Determine whether or not each eigenvalue is positive.
  def checkEigs(eig):
      # Cast to an ee Number.
      eig = ee.Number(eig)
      check = ee.Algorithms.If(eig.lt(0),eig.multiply(-1),eig.multiply(1))
      return check

  # All eigenvalues should now be positive.
  eigenVals = eigenVals.map(checkEigs)

  # Determine square root of all eigenvalues to obtain the singular values.
  singvalue = ee.Array(eigenVals).sqrt()

  def formatData(rows,value):
    new_list = ee.List([])

    # Slice array and multiply by each row's respective positive eigenvalue.
    for i in band_list_forLoops:
      sliver = ee.Algorithms.If(ee.Number(i).eq(band_list_length - 1), rows.slice(0,i).multiply(value.get([i])),
                                                    rows.slice(0,i,i+1).multiply(value.get([i])))
      # Push formatted rows into new empty list.
      new_list = new_list.add(sliver)
    return new_list

  # Get our formatted information.
  pcaload = formatData(eigenVecs,singvalue)

  # Select the top three principal components for the rotation.
  pc1 = pcaload.getArray(0).project([1]).toList()
  pc2 = pcaload.getArray(1).project([1]).toList()
  pc3 = pcaload.getArray(2).project([1]).toList()
  # Second batch of PCs.
  pc4 = pcaload.getArray(3).project([1]).toList()
  pc5 = pcaload.getArray(4).project([1]).toList()
  pc6 = pcaload.getArray(5).project([1]).toList()


  # Create the input arrays for rotation.
  varimaxInput_1 = ee.Array([pc1,pc2,pc3]).transpose()
  varimaxInput_2 = ee.Array([pc4,pc5,pc6]).transpose()

  # THIS IS WHERE I USED TO HAVE THE VPCA FUNCTION.

  # Run the varimax on PCs 1-3, then PCs 4-6.
  vpca_set1 = varimax_rotation(varimaxInput_1)
  vpca_set2 = varimax_rotation(varimaxInput_2)

  # Retrieves the resultant array from the dictionary output.
  batch1 = ee.Array(vpca_set1.get('result'))
  batch2 = ee.Array(vpca_set2.get('result'))

  # Project original data onto component axis to create component scores.
  batch1_scores = create_scores(batch1)
  batch2_scores = create_scores(batch2)
  all_scores = ee.Array.cat([batch1_scores,batch2_scores],1)

  # Cast "b" into an Image object. This will be used for the matrix multiplication.
  b_img = ee.Image(all_scores)
  # Convert the derivative Image to an Array Image.
  derivative_array = ee.Image(image).toArray(0).toArray(1)
  # Transpose the Array Image.
  derivative_array = derivative_array.arrayTranspose()
  # Dot product of Array Image and "b" Image.
  applied_b = derivative_array.matrixMultiply(b_img)
  # Transpose the array. This is an optional step to project the bands onto a column.
  applied_b = applied_b.arrayTranspose()
  # newImage = ee.Image(applied_b)

  # Get "h" for the stats output, raise to 2nd power.
  communality_1 = ee.Array(vpca_set1.get('h')).pow(2)
  communality_2 = ee.Array(vpca_set2.get('h')).pow(2)

  # Get fractions of variance.
  frac_var_set1 = ee.Array(vpca_set1.get('fracVar'))
  frac_var_set2 = ee.Array(vpca_set2.get('fracVar'))
  joined_frac_var = ee.Array.cat([frac_var_set1,frac_var_set2],1)

  # ===== #
  """This code block sorts the principal components using the fraction
     of variance array."""

  # Combine the two sets of vpca results.
  combo = ee.Array.cat([batch1,batch2],1)
  # Append the fractions of variance to the bottom of the stack.
  useToSort = ee.Array.cat([combo, joined_frac_var],0)

  # Use the fractions of variance to sort the vpca results in ascending order.
  bottomVals = useToSort.slice(0,band_list_length)
  sorted = useToSort.sort(bottomVals)

  # Flip our components to descending order.
  finalComponents = (ee.Array.cat(
    arrays = [
              sorted.slice(1,5),
              sorted.slice(1,4,5),
              sorted.slice(1,3,4),
              sorted.slice(1,2,3),
              sorted.slice(1,1,2),
              sorted.slice(1,0,1)
              ],
    axis = 1))
  # ===== #

  # Project fractions of variance down to a one-dimensional array.
  frac_var_flat = finalComponents.slice(0,band_list_length).project([1])

  # Step 1: compute the sum of the frac_var array.
  frac_var_flat_total = frac_var_flat.reduce(ee.Reducer.sum(),[0])

  # Step 2: extract the single member of the "frac_var_flat_total" array.
  frac_var_flat_total = ee.Number(frac_var_flat_total.get([0]))

  # Step 3: divide original frac_var_flat by our total observed variance.
  ex_frac_var_flat = frac_var_flat.divide(frac_var_flat_total)

  # Step 4: compute the sum of the extracted frac_var array. Should be close to 1.0.
  ex_frac_var_flat_tot = ex_frac_var_flat.reduce(ee.Reducer.sum(),[0])

  # Get PC bands, save fraction of full image variance score to each. This is needed to sort the bands.
  one = ee.Image(applied_b.arrayProject([0]).arrayGet(0)).set('frac_score',frac_var_flat.get([0]))
  two = ee.Image(applied_b.arrayProject([0]).arrayGet(1)).set('frac_score',frac_var_flat.get([1]))
  three = ee.Image(applied_b.arrayProject([0]).arrayGet(2)).set('frac_score',frac_var_flat.get([2]))
  four = ee.Image(applied_b.arrayProject([0]).arrayGet(3)).set('frac_score',frac_var_flat.get([3]))
  five = ee.Image(applied_b.arrayProject([0]).arrayGet(4)).set('frac_score',frac_var_flat.get([4]))
  six = ee.Image(applied_b.arrayProject([0]).arrayGet(5)).set('frac_score',frac_var_flat.get([5]))

  # Push PC bands into image collection, sort by frac_score in descending order, flatten into a single image and rename the bands.
  finalImage = ee.ImageCollection([one,two,three,four,five,six]).sort('frac_score', False).toBands().rename(['PC_1','PC_2','PC_3','PC_4','PC_5','PC_6'])

  # Step 5: save the extracted fractions of variance, along with their total, to the image itself.
  finalImage = finalImage.set({
    'extracted_fractions_of_variance': ex_frac_var_flat,
    'total_variance': ex_frac_var_flat_tot.get([0])
  })

  # Final Component Loadings file compiled.
  communality_tot = communality_1.add(communality_2)

  finalLoadings = ee.Array.cat([
              wl.reshape([band_list_length,1]),
              finalComponents.slice(0,0,band_list_length),
              communality_1.reshape([band_list_length,1]),
              communality_2.reshape([band_list_length,1]),
              communality_tot.reshape([band_list_length,1])],1)

  # More efficient way to double the array.
  # wl_dub = ee.Array.cat([wl,wl], axis = 0).reshape([band_list_length,1])

  # communality_tot = communality_1.add(communality_2)

  # Bring together our final array for export.
  #finalLoadings = ee.Array.cat([
  #              wl_dub,
  #              finalComponents.slice(0,0,band_list_length),
  #              communality_1.reshape([band_list_length,1]),
  #              communality_2.reshape([band_list_length,1]),
  #              communality_tot.reshape([band_list_length,1])],1)
  # Convert array to a Feature Collection; this is a necessity for Earth Engine's export functionality.
  finLoad_export = ee.FeatureCollection([createFeats_loads(finalLoadings.slice(0, i, i+1)) for i in (band_list_forLoops)])

  # These are our non-loading stats to export.
  supplementary_export = ee.FeatureCollection([
      createFeats_eigens(eigenVals, 'Eigenvalues'),
      createFeats_fracVar(frac_var_flat, 'Frac Var'),
      createFeats_fracVar(ex_frac_var_flat, 'Extracted Frac Var')])

  # Merge our two feature collections.
  finalFeatColl = finLoad_export.merge(supplementary_export)

  # Prepare the export of the VPCA loadings.
  componentsExport = batch.Export.table.toDrive(
    collection = finalFeatColl,
    description = f'6VPCA_{out_name}_finalStats_{bandCombo}',
    selectors = ['Wavelengths',
                 'component 1',
                 'component 2',
                 'component 3',
                 'component 4',
                 'component 5',
                 'component 6',
                 'Comm_1-3',
                 'Comm_4-6',
                 'Comm_tot'])

  # Initialize the export.
  batch.Task.start(componentsExport)

  # Monitor the task.
  i = 1
  while componentsExport.status()['state'] in ['READY', 'RUNNING']:
    sys.stdout.write("\r" + f'Final stats export status update #{i}: ' + str(componentsExport.status()))
    sys.stdout.flush()
    i += 1
    time.sleep(5)

  else:
    print('Final stats export completed...')
    print(componentsExport.status())
    time.sleep(10)

  # We're moving our file to the desired destination file.
  # Output name.
  out = f'{output_folder}/{out_fold}'
  print(out)
  try:
    # If the file already exists, get rid of original and replace with the new file.
    if os.path.exists(f"{out}/6VPCA_{out_name}_finalStats_{bandCombo}.csv"):
      os.remove(f"{out}/6VPCA_{out_name}_finalStats_{bandCombo}.csv")
      # Found here: https://stackoverflow.com/questions/51109931/moving-files-in-google-colab
      shutil.move(f'/content/gdrive/My Drive/6VPCA_{out_name}_finalStats_{bandCombo}.csv', out)
    else:
      shutil.move(f'/content/gdrive/My Drive/6VPCA_{out_name}_finalStats_{bandCombo}.csv', out)

  # Let it sleep for a second to let Google Drive catch up.
  except:
    time.sleep(10)

    if os.path.exists(f"{out}/6VPCA_{out_name}_finalStats_{bandCombo}.csv"):
      os.remove(f"{out}/6VPCA_{out_name}_finalStats_{bandCombo}.csv")
      shutil.move(f'/content/gdrive/My Drive/6VPCA_{out_name}_finalStats_{bandCombo}.csv', out)
    else:
      shutil.move(f'/content/gdrive/My Drive/6VPCA_{out_name}_finalStats_{bandCombo}.csv', out)

  return [finalImage, ex_frac_var_flat]

# Run analysis over all images in stack

In [ ]:
# Empty list to shove our output images into.
images = []

weighted_matrix = f'/content/gdrive/My Drive/test_FL_DEP/S2_avgCorrelationMatrix.csv'

# csv_files, der_list, image_names,output_folders
for i in range(len(der_list)):
  forSumImages = fullAnalysis(weighted_matrix, der_list[i], image_names[i], output_folders[i])
  images += [
  fullAnalysis(weighted_matrix, der_list[i], image_names[i], output_folders[i])[0]
  ]
  print(f'Image {i+1} done...')


<ipython-input-24-19c2b595e5de>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dataframe = pd.read_csv(drive_string, delimiter = ',').drop('Unnamed: 0',1)


Final stats export status update #2: {'state': 'READY', 'description': '6VPCA_20210719T160519_20210719T160513_T17RNL_finalStats_B1-B8A', 'creation_timestamp_ms': 1683592987490, 'update_timestamp_ms': 1683592987490, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'YRH6YBCIDEVZRFL5J5NLRQNL', 'name': 'projects/earthengine-legacy/operations/YRH6YBCIDEVZRFL5J5NLRQNL'}Final stats export completed...
{'state': 'COMPLETED', 'description': '6VPCA_20210719T160519_20210719T160513_T17RNL_finalStats_B1-B8A', 'creation_timestamp_ms': 1683592987490, 'update_timestamp_ms': 1683592996922, 'start_timestamp_ms': 1683592996178, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.0014739319449290633, 'id': 'YRH6YBCIDEVZRFL5J5NLRQNL', 'name': 'projects/earthengine-legacy/operations/YRH6YBCIDEVZRFL5J5NLRQNL'}
/content/gdrive/My Drive/test_FL_DEP/part2/20200101T160511_20200101T160506_T17RNL_outputs


<ipython-input-24-19c2b595e5de>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dataframe = pd.read_csv(drive_string, delimiter = ',').drop('Unnamed: 0',1)


Final stats export status update #2: {'state': 'READY', 'description': '6VPCA_20210719T160519_20210719T160513_T17RNL_finalStats_B1-B8A', 'creation_timestamp_ms': 1683593010146, 'update_timestamp_ms': 1683593010146, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'SQ5AF44NTGUR7QRR5J6JJ5DY', 'name': 'projects/earthengine-legacy/operations/SQ5AF44NTGUR7QRR5J6JJ5DY'}Final stats export completed...
{'state': 'COMPLETED', 'description': '6VPCA_20210719T160519_20210719T160513_T17RNL_finalStats_B1-B8A', 'creation_timestamp_ms': 1683593010146, 'update_timestamp_ms': 1683593018877, 'start_timestamp_ms': 1683593017529, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.0007409839890897274, 'id': 'SQ5AF44NTGUR7QRR5J6JJ5DY', 'name': 'projects/earthengine-legacy/operations/SQ5AF44NTGUR7QRR5J6JJ5DY'}
/content/gdrive/My Drive/test_FL_DEP/part2/20200101T160511_20200101T160506_T17RNL_outputs
Image 1 done...


# Image Exports

In [ ]:
# JOE: I am saving colormaps to variables here.
# https://stackoverflow.com/questions/33596491/extract-matplotlib-colormap-in-hex-format

def hex_colormap(palette_name = None):
  # "palette_name" must be a string.
  ramp = cm.get_cmap(palette_name)
  # Converts color values to hexadecimal format so we can use them.
  ramp_ls = [mpl.colors.rgb2hex(ramp(color)[:3]) for color in range(ramp.N)]

  return ramp_ls
# See a full list of colormaps here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
#perceptually uniform colormps
viridis = hex_colormap('viridis')
inferno = hex_colormap('inferno')
magma = hex_colormap('magma')
gist_gray = hex_colormap('gist_gray')

#divergent (Add _r to invert the color ramp)
BrBG = hex_colormap('BrBG')
spectral = hex_colormap('Spectral')
coolwarm = hex_colormap('coolwarm')
seismic = hex_colormap('seismic')

BrBG_r = hex_colormap('BrBG_r')
spectral_r = hex_colormap('Spectral_r')
coolwarm_r = hex_colormap('coolwarm_r')
seismic_r = hex_colormap('seismic_r')

# Function to find the tails of the pdf
# This returns the tails of the pdf for the image with values from p0-p10, and p90-p100
# This allows the user to select a range of min-max values as plotting limits

def minAndMax(image,region = None):

  # Find the percentile values using a reduceRegion call.
  min_reducer = image.reduceRegion(
      reducer = ee.Reducer.percentile([0,1,2,3,4,5,6,7,8,9,10,90,91,92,93,94,95,96,97,98,99,100],['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p90','p91','p92','p93','p94','p95','p96','p97','p98','p99','p100']),
          scale = 10,
          geometry = region,
          bestEffort = True)

  orig_keys = min_reducer.keys()

  min_reducer = min_reducer.rename([orig_keys.get(0),orig_keys.get(1),orig_keys.get(2),orig_keys.get(3),orig_keys.get(4),orig_keys.get(5),orig_keys.get(6),orig_keys.get(7),orig_keys.get(8),orig_keys.get(9),orig_keys.get(10),orig_keys.get(11),orig_keys.get(12),orig_keys.get(13),orig_keys.get(14),orig_keys.get(15),orig_keys.get(16),orig_keys.get(17),orig_keys.get(18),orig_keys.get(19),orig_keys.get(20),orig_keys.get(21)],['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p90','p91','p92','p93','p94','p95','p96','p97','p98','p99','p100'])

  return min_reducer

<ipython-input-27-00238459f61e>:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  ramp = cm.get_cmap(palette_name)


In [ ]:
 #pc1_vals = ee.List([])
 #pc2_vals = ee.List([])
 #pc3_vals = ee.List([])
 #pc4_vals = ee.List([])
 #pc5_vals = ee.List([])
 #pc6_vals = ee.List([])

 #for elem in range(len(images)):
   # Convert element to ee.Image.
 #  image = ee.Image(images[elem])

   # Get min and max values for each principal component, then select out their 2% and 98% stretches.
  # minMax_pc1 = minAndMax(image.select('PC_1'), roi)
  # pc1_min = minMax_pc1.get('p2')
  # pc1_max = minMax_pc1.get('p98')
  # pc1_vals = pc1_vals.cat([pc1_min, pc1_max])

  # minMax_pc2 = minAndMax(image.select('PC_2'), roi)
  # pc2_min = minMax_pc2.get('p2')
  # pc2_max = minMax_pc2.get('p98')
  # pc2_vals = pc2_vals.cat([pc2_min, pc2_max])

  # minMax_pc3 = minAndMax(image.select('PC_3'), roi)
  # pc3_min = minMax_pc3.get('p2')
  # pc3_max = minMax_pc3.get('p98')
  # pc3_vals = pc3_vals.cat([pc3_min, pc3_max])

  # minMax_pc4 = minAndMax(image.select('PC_4'), roi)
  # pc4_min = minMax_pc4.get('p2')
  # pc4_max = minMax_pc4.get('p98')
  # pc4_vals = pc4_vals.cat([pc4_min, pc4_max])

  # minMax_pc5 = minAndMax(image.select('PC_5'), roi)
  # pc5_min = minMax_pc5.get('p2')
  # pc5_max = minMax_pc5.get('p98')
  # pc5_vals = pc5_vals.cat([pc5_min, pc5_max])

  # minMax_pc6 = minAndMax(image.select('PC_6'), roi)
  # pc6_min = minMax_pc6.get('p2')
  # pc6_max = minMax_pc6.get('p98')
  # pc6_vals = pc6_vals.cat([pc6_min, pc6_max])

In [ ]:
# Conversion to numpy values. This needs to be done because the EE min/max values
# need to be reterieved in order to call them as values in the next block.
 # pc1_vals_np = np.array(pc1_vals.getInfo())
 # pc1_min = np.percentile(pc1_vals_np,2)
 # pc1_max = np.percentile(pc1_vals_np,98)

 # pc2_vals_np = np.array(pc2_vals.getInfo())
 # pc2_min = np.percentile(pc2_vals_np,2)
 # pc2_max = np.percentile(pc2_vals_np,98)

 # pc3_vals_np = np.array(pc3_vals.getInfo())
 # pc3_min = np.percentile(pc3_vals_np,2)
 # pc3_max = np.percentile(pc3_vals_np,98)

 # pc4_vals_np = np.array(pc4_vals.getInfo())
 # pc4_min = np.percentile(pc4_vals_np,2)
 # pc4_max = np.percentile(pc4_vals_np,98)

 # pc5_vals_np = np.array(pc5_vals.getInfo())
 # pc5_min = np.percentile(pc5_vals_np,2)
 # pc5_max = np.percentile(pc5_vals_np,98)

 # pc6_vals_np = np.array(pc6_vals.getInfo())
 # pc6_min = np.percentile(pc6_vals_np,2)
 # pc6_max = np.percentile(pc6_vals_np,98)

In [ ]:
# print(pc6_max)

In [ ]:
def export_image(image, out_name, out_fold):
  # Create visualizations of our output image bands. This reduces the file size considerably.
  exportViz_pc1 = image.select('PC_1')
  exportViz_pc2 = image.select('PC_2')
  exportViz_pc3 = image.select('PC_3')
  exportViz_pc4 = image.select('PC_4')
  exportViz_pc5 = image.select('PC_5')
  exportViz_pc6 = image.select('PC_6')

  # Put the bands into a list, so we can loop over the list to start the exports.
  exp_ls = [exportViz_pc1,
            exportViz_pc2,
            exportViz_pc3,
            exportViz_pc4,
            exportViz_pc5,
            exportViz_pc6]

  j = 1
  for im in exp_ls:
    export = batch.Export.image.toDrive(
      image = im,
      region = roi,
      folder = 'test_FL_DEP',
      description = f'6VPCA_{out_name}_PC{j}',
      scale = native_res,
      crs = 'EPSG:4326',
      maxPixels = maxP)

    # Initialize the export.
    batch.Task.start(export)

    j+=1

  return

for elem in range(len(images)):
    export_image(ee.Image(images[elem]), image_names[elem], output_folders[elem])

# Export an RGB and each PC as its own raster

Generate raster for RGB image

In [ ]:
# Add code here to save the RBG three band composite to disk
def rgb(im, out_name):
  im = ee.Image(im)
  rgb_minMax = minAndMax(im.select('B3'),roi)

  rgb_viz = (im.select(['B4', 'B3', 'B2']).visualize(
             min = rgb_minMax.get('p5'),
             max = rgb_minMax.get('p95')))

  rgb_out = batch.Export.image.toDrive(
    image = rgb_viz,
    region = roi,
    folder = 's2_outputs',
    description = f'6VPCA_{out_name}_rgb',
    scale = native_res,
    crs = 'EPSG:4326',
    maxPixels = maxP)

  batch.Task.start(rgb_out)

for elem in range(len(images)):
  rgb(im_list[elem],image_names[elem])

# Extract raw files for thresholding

In [ ]:
# def export_image_raw(image, out_name, out_fold):

#   exportViz_pc1 = image.select('PC_1')
#   exportViz_pc2 = image.select('PC_2')
#   exportViz_pc3 = image.select('PC_3')
#   exportViz_pc4 = image.select('PC_4')
#   exportViz_pc5 = image.select('PC_5')
#   exportViz_pc6 = image.select('PC_6')

#   exp_ls = [
#             exportViz_pc1,
#             exportViz_pc2,
#             exportViz_pc3,
#             exportViz_pc4,
#             exportViz_pc5,
#             exportViz_pc6
#             ]

#   j = 1
#   for im in exp_ls:
#     export = batch.Export.image.toDrive(
#       image = im,
#       region = roi,
#       folder = 's2_outputs_raw',
#       description = f'6VPCA_{out_name}_PC{j}_rawFile',
#       scale = native_res,
#       crs = 'EPSG:4326',
#       maxPixels = maxP)

#     # Initialize the export.
#     batch.Task.start(export)

#     j+=1

#   return

# for elem in range(len(images)):
#   export_image_raw(ee.Image(images[elem]), image_names[elem], output_folders[elem])

## Code to calculate the weighted sum of the 3 leading components



In [ ]:
def weighted_pcs3(im, out_name):
  pc_sum3 = im.expression(
      '(exVar_3*pc_3)', {
      'exVar_3' : ee.Array(forSumImages[1]).get([0]),
      'pc_3': ee.Image(forSumImages[0]).select('PC_3'),
  })

  # now generate an output map

  pc_sum3_minMax = minAndMax(pc_sum3,roi)

  pc_sum3_viz = (pc_sum3.visualize(palette = spectral,
             min = pc_sum3_minMax.get('p1'),
             max = pc_sum3_minMax.get('p99')))

  vpc_sum3 = batch.Export.image.toDrive(
    image = pc_sum3_viz,
    region = roi,
    description = f'6VPCA_{out_name}_C3Viz',
    folder = 'Component_Viz',
    scale = 10,
    crs = 'EPSG:4326',
    maxPixels = maxP)

  batch.Task.start(vpc_sum3)

for elem in range(len(images)):
  weighted_pcs3(ee.Image(images[elem]),image_names[elem])

In [ ]:
# Now calculate the sum of the three VPC images to get a thematic image with noise removed
# Each principal component is rescaled by its extracted variance
def weighted_pcs1to3(im, out_name):
  pc_sum3 = im.expression(
      '(exVar_1*pc_1 + exVar_2*pc_2 + exVar_3*pc_3)', {
      'exVar_1' : ee.Array(forSumImages[1]).get([0]),
      'exVar_2' : ee.Array(forSumImages[1]).get([1]),
      'exVar_3' : ee.Array(forSumImages[1]).get([2]),
      'pc_1': ee.Image(forSumImages[0]).select('PC_1'),
      'pc_2': ee.Image(forSumImages[0]).select('PC_2'),
      'pc_3': ee.Image(forSumImages[0]).select('PC_3')
  })

  # now generate an output map

  pc_sum3_minMax = minAndMax(pc_sum3,roi)

  pc_sum3_viz = (pc_sum3.visualize(palette = spectral,
             min = pc_sum3_minMax.get('p2'),
             max = pc_sum3_minMax.get('p98')))

  vpc_sum3 = batch.Export.image.toDrive(
    image = pc_sum3_viz,
    region = roi,
    description = f'6VPCA_{out_name}_sum3',
    scale = 10,
    crs = 'EPSG:4326',
    maxPixels = maxP)

  batch.Task.start(vpc_sum3)

for elem in range(len(images)):
  weighted_pcs1to3(ee.Image(images[elem]),image_names[elem])

## Code to calculate the weighted sum of the 6 extracted components


In [ ]:
# Now calculate the sum of the six extracted VPC images to get a thematic image with noise removed
# Each principal component is rescaled by its extracted variance
def weighted_pcs1to6(im, out_name):
  pc_sum6 = finalImage.expression(
      '(exVar_1*pc_1 + exVar_2*pc_2 + exVar_3*pc_3 + exVar_4*pc_4 + exVar_5*pc_5 + exVar_6*pc_6)', {
      'exVar_1' : ex_frac_var_flat.get([0]),
      'exVar_2' : ex_frac_var_flat.get([1]),
      'exVar_3' : ex_frac_var_flat.get([2]),
      'exVar_4' : ex_frac_var_flat.get([3]),
      'exVar_5' : ex_frac_var_flat.get([4]),
      'exVar_6' : ex_frac_var_flat.get([5]),
      'pc_1': finalImage.select('PC_1'),
      'pc_2': finalImage.select('PC_2'),
      'pc_3': finalImage.select('PC_3'),
      'pc_4': finalImage.select('PC_4'),
      'pc_5': finalImage.select('PC_5'),
      'pc_6': finalImage.select('PC_6')
  })

  # now generate an output map
  pc_sum6_minMax = minAndMax(pc_sum6,roi)

  pc_sum6_viz = (pc_sum6.visualize(palette = spectral,
             min = pc_sum6_minMax.get('p2'),
             max = pc_sum6_minMax.get('p98')))

  vpc_sum6 = batch.Export.image.toDrive(
    image = pc_sum6_viz,
    region = water_geom.geometry().bounds().getInfo()['coordinates'],
    folder = 'KSU_VPCA_Outputs',
    description = f'6VPCA_{out_name}_sum6',
    scale = 30,
    crs = 'EPSG:4326',
    maxPixels = 1e8)
  # Comment out line below to prevent write of file to disk
  batch.Task.start(vpc_sum6)

for elem in range(len(images)):
  weighted_pcs1to3(ee.Image(images[elem]),image_names[elem])